<H1> Example on using the datasets for quantum state tomography </H1>

In [ ]:
# preample
import zipfile
import pickle
import os
import numpy as np

In [ ]:
'''The datalist below lists each different dataset category.
Each category comprises two compressed files:
(a) one for the non-distorted examples; and
(b) another for the distorted examples.

Each dataset is stored in a folder corresponding to its category on Cloudstor.
There are 52 datasets in the QDataSet and 26 categories.


To run the code:
1. Download both the distorted and non-distorted datasets from Cloudstor
(assuming both are to be used);
2. To select a dataset for the model (e.g. G_1q_X), uncomment the dataset
in the dataset list below.'''

In [ ]:
datalist = [
'G_1q_X',
# 'G_1q_XY',
# 'G_1q_XY_XZ_N1N5',
# 'G_1q_XY_XZ_N1N6',
# 'G_1q_XY_XZ_N3N6',
# 'G_1q_X_Z_N1',
# 'G_1q_X_Z_N2',
# 'G_1q_X_Z_N3',
# 'G_1q_X_Z_N4',
# 'G_2q_IX-XI_IZ-ZI_N1-N6',
# 'G_2q_IX-XI-XX',
# 'G_2q_IX-XI-XX_IZ-ZI_N1-N5',
# 'G_2q_IX-XI-XX_IZ-ZI_N1-N5',
# 'S_1q_X',
# 'S_1q_XY',
# 'S_1q_XY_XZ_N1N5',
# 'S_1q_XY_XZ_N1N6',
# 'S_1q_XY_XZ_N3N6',
# 'S_1q_X_Z_N1',
# 'S_1q_X_Z_N2',
# 'S_1q_X_Z_N3',
# 'S_1q_X_Z_N4',
# 'S_2q_IX-XI_IZ-ZI_N1-N6',
# 'S_2q_IX-XI-XX',
# 'S_2q_IX-XI-XX_IZ-ZI_N1-N5',
# 'S_2q_IX-XI-XX_IZ-ZI_N1-N6',
]

In [ ]:
'''Create two strings, one for each of the undistorted and distorted datasets.'''
data1 = datalist[0]
data2 = data1 + '_D'
print(data1)
print(data2)

S_2q_IX-XI-XX_IZ-ZI_N1-N5
S_2q_IX-XI-XX_IZ-ZI_N1-N5_D


In [ ]:
'''Set the working directory to the location of the datasets, example below.'''
os.chdir('/projects/QuantumDS/QDataSet/' + data1)

## Pure state tomography

In [ ]:
'''Define a function to extract the data from the dataset.'''

def load_pure_dataset(dataset_name, num_training_ex, num_testing_ex):
    
    # initialize empt lists for storing the data
    data_input         = []
    data_target        = []
    
    # 1) unzip the dataset zipfile
    fzip               = zipfile.ZipFile("%s.zip"%dataset_name, mode='r')
    
    # loop over example files
    #########################################################
    for idx_ex in range(num_training_ex + num_testing_ex):
    
        # 2) extract the example file from the dataset 
        fname = "%s_ex_%d"%(dataset_name, idx_ex)
        fzip.extract( fname )
    
        # 3) load the example file
        f     = open(fname,  "rb")
        data  = pickle.load(f)
    
        # 4) extract the useful information from the example file:
        
        # For state tomograhpy we need to extract a set of states and the corresponding Pauli observables,
        # so we pick arbitrarily the evolved state at t=T from an initial X+ state, and the first noise realization.
        # We can choose initial state/ time / realization
        
        idx_time          = 1023 # t=T
        idx_realization   = 0    # first realization
        idx_initial_state = 0    # Pauli X+ initial state
        
        # construct the full unitary from the product of control unitary and interaction unitary
        U = data["U0"][0,idx_time,0,:]@data["UI"][0,idx_realization,:]
        
        # evolve the initial state
        rho_0             = data["sim_parameters"]["initial_states"][idx_initial_state] # initial state
        rho_t             = U @ rho_0 @ U.conj().T                                      # evolved state
        
        # extract corresponding Pauli observables
        measurements      = data["Eo"][0:1,idx_realization,idx_initial_state:idx_initial_state+3]
        
        # add the pair of input and target to the corresponding lists
        
        data_input.append( measurements )
        data_target.append( np.reshape(rho_t, (1,rho_t.shape[0],rho_t.shape[1])) )
        
        # 5) close and delete the example file
        f.close()
        os.remove(fname)
    #########################################################
    # 5) close the dataset zipfile
    fzip.close()
    
    # 6) split the data into training and testing
    data_input            = np.concatenate(data_input, axis=0)
    data_target           = np.concatenate(data_target, axis=0)
    
    training_input        = data_input[0:num_training_ex, :]
    testing_input         = data_input[num_training_ex:num_training_ex+num_testing_ex, :]
    
    training_target       = data_target[0:num_training_ex, :]
    testing_target        = data_target[num_training_ex:num_training_ex+num_testing_ex, :]
    
    return training_input, training_target, testing_input, testing_target

In [ ]:
# define the dataset parameters
dataset_name    = data1 #"G_1q_XY_XZ_N1N5" # dataset name
num_training_ex = 7                 # number of training examples
num_testing_ex  = 3                 # number of testing examples

In [ ]:
# load the dataset
training_input, training_target, testing_input, testing_target = load_pure_dataset(dataset_name, num_training_ex, num_testing_ex)

In [ ]:
print(training_input.shape, training_target.shape, testing_input.shape, testing_target.shape)

(7, 3) (7, 4, 4) (3, 3) (3, 4, 4)


In [ ]:
'''Set input parameter shape based on number of axes along which controls / noise is applied.'''
axnum = training_input.shape[1]
axnum2 = training_target.shape[1]


3 4


In [ ]:
# use the dataset for training and testing an ML algorithm

#############################################
##          PUT YOUR CODE HERE             ##

# trained_model = my_training_function(training_input, training_target)

# performance   = my_testing_function(trained_model, testing_input, testing_target)

### Below is an example using tensorflow  ###

import tensorflow.keras as K

input_layer   = K.Input(shape=(axnum))

output_layer  = K.layers.Reshape((axnum2,axnum2))(K.layers.Dense(axnum2**2)(input_layer))

ml_model      = K.Model(input_layer, output_layer)

ml_model.compile(optimizer=K.optimizers.Adam(), loss='mse')

ml_model.fit(training_input, training_target, epochs=10, validation_data = (testing_input, testing_target))

Epoch 1/10
1/1 [==============================] - 0s 227ms/step - loss: 0.0971 - val_loss: 0.1189
Epoch 2/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0965 - val_loss: 0.1182
Epoch 3/10
1/1 [==============================] - 0s 11ms/step - loss: 0.0959 - val_loss: 0.1174
Epoch 4/10
1/1 [==============================] - 0s 11ms/step - loss: 0.0954 - val_loss: 0.1167
Epoch 5/10
1/1 [==============================] - 0s 11ms/step - loss: 0.0948 - val_loss: 0.1160
Epoch 6/10
1/1 [==============================] - 0s 11ms/step - loss: 0.0943 - val_loss: 0.1153
Epoch 7/10
1/1 [==============================] - 0s 11ms/step - loss: 0.0938 - val_loss: 0.1146
Epoch 8/10
1/1 [==============================] - 0s 11ms/step - loss: 0.0932 - val_loss: 0.1139
Epoch 9/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0927 - val_loss: 0.1132
Epoch 10/10
1/1 [==============================] - 0s 13ms/step - loss: 0.0922 - val_loss: 0.1125


## Mixed state tomography

In [ ]:
def load_mixed_dataset(dataset_name, num_training_ex, num_testing_ex):
    
    # initialize empt lists for storing the data
    data_input         = []
    data_target        = []
    
    # 1) unzip the dataset zipfile
    fzip               = zipfile.ZipFile("%s.zip"%dataset_name, mode='r')
    
    # loop over example files
    #########################################################
    for idx_ex in range(num_training_ex + num_testing_ex):
    
        # 2) extract the example file from the dataset 
        fname = "%s_ex_%d"%(dataset_name, idx_ex)
        fzip.extract( fname )
    
        # 3) load the example file
        f     = open(fname,  "rb")
        data  = pickle.load(f)
    
        # 4) extract the useful information from the example file:
        
        # For state tomograhpy we need to extract a set of states and the corresponding Pauli observables,
        # so we pick arbitrarily the evolved state at t=T from an initial X+ state, and averaged over all noize realizations.
        # We can choose initial state/ time / realization
        
        idx_time          = 1023 # t=T
        idx_initial_state = 0    # Pauli X+ initial state index
        
        rho_0             = data["sim_parameters"]["initial_states"][idx_initial_state] # initial state
        U0                = data["U0"][0,idx_time,0,:]                                  # control unitary
        
        
        
        rho_t             = [] # list of evolved states at t=T for every noise realization 
        measurements      = [] # list of measurements at t=T for every noise realization 
        
        # loop over noise realizations
        for idx_UI, UI in enumerate(data["UI"][0,:,0:U0.shape[0], 0:U0.shape[1]]):  
            # evolve the initial state
            rho_t.append( (U0@UI) @ rho_0 @ (U0@UI).conj().T )
        
            # extract corresponding Pauli observables
            measurements.append( data["Eo"][0:1, idx_UI, idx_initial_state:idx_initial_state+3] )
        
        # average over all noise realizations to get a mixed state
        rho_t             = sum(rho_t) / data["sim_parameters"]["K"]
        measurements      = sum(measurements) / data["sim_parameters"]["K"]
        
        # add the pair of input and target to the corresponding lists
        data_input.append( measurements )
        data_target.append( np.reshape(rho_t, (1,rho_t.shape[0],rho_t.shape[1])) )
        
        # 5) close and delete the example file
        f.close()
        os.remove(fname)
    #########################################################
    # 5) close the dataset zipfile
    fzip.close()
    
    # 6) split the data into training and testing
    data_input            = np.concatenate(data_input, axis=0)
    data_target           = np.concatenate(data_target, axis=0)
    
    training_input        = data_input[0:num_training_ex, :]
    testing_input         = data_input[num_training_ex:num_training_ex+num_testing_ex, :]
    
    training_target       = data_target[0:num_training_ex, :]
    testing_target        = data_target[num_training_ex:num_training_ex+num_testing_ex, :]
    
    return training_input, training_target, testing_input, testing_target

In [ ]:
# define the dataset parameters
dataset_name    = data1 #"G_1q_XY_XZ_N1N5" # dataset name
num_training_ex = 7                 # number of training examples
num_testing_ex  = 3                 # number of testing examples

In [ ]:
# load the dataset
training_input, training_target, testing_input, testing_target = load_mixed_dataset(dataset_name, num_training_ex, num_testing_ex)

In [ ]:
print(training_input.shape, training_target.shape, testing_input.shape, testing_target.shape)

(7, 3) (7, 4, 4) (3, 3) (3, 4, 4)


In [ ]:
'''Set input parameter shape based on number of axes along which controls / noise is applied.'''
axnum = training_input.shape[1]
axnum2 = training_target.shape[1]


3 4


In [ ]:
# use the dataset for training and testing an ML algorithm

#############################################
##          PUT YOUR CODE HERE             ##

# trained_model = my_training_function(training_input, training_target)

# performance   = my_testing_function(trained_model, testing_input, testing_target)

### Below is an example using tensorflow  ###

import tensorflow.keras as K

input_layer   = K.Input(shape=(axnum))

output_layer  = K.layers.Reshape((axnum2,axnum2))(K.layers.Dense(axnum2**2)(input_layer))

ml_model      = K.Model(input_layer, output_layer)

ml_model.compile(optimizer=K.optimizers.Adam(), loss='mse')

ml_model.fit(training_input, training_target, epochs=10, validation_data = (testing_input, testing_target))

Epoch 1/10
1/1 [==============================] - 0s 241ms/step - loss: 0.0689 - val_loss: 0.0854
Epoch 2/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0684 - val_loss: 0.0849
Epoch 3/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0680 - val_loss: 0.0844
Epoch 4/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0675 - val_loss: 0.0838
Epoch 5/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0670 - val_loss: 0.0833
Epoch 6/10
1/1 [==============================] - 0s 13ms/step - loss: 0.0666 - val_loss: 0.0828
Epoch 7/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0661 - val_loss: 0.0823
Epoch 8/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0657 - val_loss: 0.0818
Epoch 9/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0653 - val_loss: 0.0813
Epoch 10/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0648 - val_loss: 0.0808


## Model the effect of sampling noise (finite number of shots)

In [ ]:
Nshots = 1024 # Number of shots

In [ ]:
training_input_noisy = np.zeros(training_input.shape)
testing_input_noisy  = np.zeros(testing_input.shape)

# generate a binomial distribution with mean centered at the ideal expectation values, the states (targets) do not change
for idx_ex in range(num_training_ex):
    training_input_noisy[idx_ex,:] = [(2*np.random.binomial(Nshots, 0.5*(exp+1) ) - Nshots)/Nshots for exp in training_input[idx_ex,:]]
    
for idx_ex in range(num_testing_ex):
    testing_input_noisy[idx_ex,:] = [(2*np.random.binomial(Nshots, 0.5*(exp+1) ) - Nshots)/Nshots for exp in testing_input[idx_ex,:]]

In [ ]:
# use the dataset for training and testing an ML algorithm

#############################################
##          PUT YOUR CODE HERE             ##

# trained_model = my_training_function(training_input_noisy, training_target)

# performance   = my_testing_function(trained_model, testing_input_noisy, testing_target)

### Below is an example using tensorflow  ###

import tensorflow.keras as K

input_layer   = K.Input(shape=(axnum))

output_layer  = K.layers.Reshape((axnum2,axnum2))(K.layers.Dense(axnum2**2)(input_layer))

ml_model      = K.Model(input_layer, output_layer)

ml_model.compile(optimizer=K.optimizers.Adam(), loss='mse')

ml_model.fit(training_input, training_target, epochs=10, validation_data = (testing_input, testing_target))

Epoch 1/10
1/1 [==============================] - 0s 224ms/step - loss: 0.0588 - val_loss: 0.0516
Epoch 2/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0584 - val_loss: 0.0511
Epoch 3/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0580 - val_loss: 0.0507
Epoch 4/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0576 - val_loss: 0.0503
Epoch 5/10
1/1 [==============================] - 0s 11ms/step - loss: 0.0571 - val_loss: 0.0498
Epoch 6/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0567 - val_loss: 0.0494
Epoch 7/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0563 - val_loss: 0.0490
Epoch 8/10
1/1 [==============================] - 0s 11ms/step - loss: 0.0559 - val_loss: 0.0486
Epoch 9/10
1/1 [==============================] - 0s 12ms/step - loss: 0.0555 - val_loss: 0.0482
Epoch 10/10
1/1 [==============================] - 0s 11ms/step - loss: 0.0551 - val_loss: 0.0478
